# Chapter 5 — Eigenvalues and Eigenvectors (Colab Companion)

This notebook provides **Python helpers and demos** for Chapter 5 topics:
- **Definitions & computation:** characteristic polynomial, eigenvalues, eigenvectors
- **Properties:** trace / determinant relations, triangular case, eigenvalues of powers and inverses
- **Algebraic vs geometric multiplicity; diagonalizability checks**
- **Leverrier–Faddeev method** for the characteristic polynomial
- **Strict diagonal dominance (SDD)** and nonsingularity (via Gershgorin)
- Clean **NumPy** demos plus **SymPy** for exact/structural checks

> All explanatory comments are in **English** to align with your request.

In [ ]:
# Setup: SymPy (symbolic) and NumPy (numeric). In Colab these are available by default.
# If needed, you can uncomment the next line to install/upgrade:
# !pip -q install sympy numpy matplotlib

import numpy as np
import sympy as sp
import math

import matplotlib.pyplot as plt
from matplotlib.patches import Circle

sp.init_printing(use_unicode=True)

def to_sym(A):
    """Coerce to a SymPy Matrix."""
    if isinstance(A, sp.MatrixBase):
        return A
    return sp.Matrix(A)

def is_square(A):
    A = to_sym(A)
    return A.shape[0] == A.shape[1]

## 1) Characteristic polynomial, eigenvalues, eigenvectors
We offer both **numeric** (NumPy) and **symbolic** (SymPy) paths.

In [ ]:
def charpoly_sym(A):
    """Characteristic polynomial of A as a SymPy Poly in λ."""
    A = to_sym(A)
    lam = sp.symbols('λ')
    return A.charpoly(lam).as_poly()

def eig_numpy(A):
    """Return (eigvals, eigvecs) from NumPy (possibly complex)."""
    A = np.array(A, dtype=float)
    vals, vecs = np.linalg.eig(A)
    return vals, vecs

def eig_sympy(A):
    """Return eigenvalues (with algebraic multiplicity) and eigenvectors (sympy)."""
    A = to_sym(A)
    # eigenvects() returns tuples (eigenvalue, multiplicity, [basis vectors])
    return A.eigenvects()

## 2) Core properties to verify
- **Sum of eigenvalues = trace(A)**; **product = det(A)** (over algebraic closure).
- Eigenvalues of a **triangular** matrix are its diagonal entries.
- If A is invertible, **eigenvalues(A^{-1}) = 1 / eigenvalues(A)**.
- For k ∈ N, **eigenvalues(A^k) = (eigenvalues(A))^k**.

In [ ]:
def verify_trace_det(A):
    A = to_sym(A)
    lam = sp.symbols('λ')
    cp = A.charpoly(lam)
    eigs = [sp.N(ev) for ev in A.eigenvals().keys()]  # unique eigenvalues (SymPy may repeat via dict)
    # For sums/products, use algebraic multiplicities
    eigs_mult = []
    for ev, mult in A.eigenvals().items():
        eigs_mult += [ev]*mult
    s = sp.simplify(sum(eigs_mult))
    p = sp.simplify(sp.prod(eigs_mult) if eigs_mult else 1)
    return {
        "trace(A)": sp.simplify(A.trace()),
        "sum_eigs": sp.simplify(s),
        "det(A)": sp.simplify(A.det()),
        "prod_eigs": sp.simplify(p),
        "charpoly": cp.as_expr()
    }

def is_triangular(A):
    A = to_sym(A)
    upper = A.equals(sp.Matrix(np.triu(np.array(A.tolist(), dtype=float))))
    lower = A.equals(sp.Matrix(np.tril(np.array(A.tolist(), dtype=float))))
    return {"upper": upper, "lower": lower}

def eigs_of_power_and_inverse(A, k=2):
    A = to_sym(A)
    if A.det() == 0:
        inv = None
    else:
        inv = A.inv()
    Ak = A**k
    return {
        "eigs(A)": list(A.eigenvals().keys()),
        f"eigs(A^{k})": list(Ak.eigenvals().keys()),
        "eigs(A^{-1})": list(inv.eigenvals().keys()) if inv is not None else None
    }

## 3) Algebraic vs. geometric multiplicity; diagonalizability
A matrix is **diagonalizable** iff the direct sum of eigenspaces has total dimension `n` (i.e., the sum of **geometric multiplicities** equals `n`).

In [ ]:
def algebraic_geometric_multiplicities(A):
    A = to_sym(A)
    n = A.shape[0]
    evals = A.eigenvals()  # dict {eigenvalue: algebraic multiplicity}
    out = []
    for lam, am in evals.items():
        geom_mult = (A - lam*sp.eye(n)).nullspace()
        out.append({
            "eigenvalue": lam,
            "alg_mult": int(am),
            "geom_mult": len(geom_mult)
        })
    diag = sum(item["geom_mult"] for item in out) == n
    return {"multiplicities": out, "diagonalizable": diag}

def diagonalize_if_possible(A):
    """Return (P, D) with A = P*D*P^{-1}, if diagonalizable; else (None, None)."""
    A = to_sym(A)
    res = algebraic_geometric_multiplicities(A)
    if not res["diagonalizable"]:
        return None, None
    # Build eigenbasis columns
    n = A.shape[0]
    cols = []
    vals = []
    for lam, am in A.eigenvals().items():
        basis = (A - lam*sp.eye(n)).nullspace()
        for v in basis:
            cols.append(v)
            vals.append(lam)
    P = sp.Matrix.hstack(*cols)
    D = sp.diag(*vals)
    if P.det() == 0:
        return None, None
    return P, D

## 4) Leverrier–Faddeev method (characteristic polynomial)
Construct the characteristic polynomial \( p(λ) = λ^n + c_1 λ^{n-1} + \cdots + c_{n-1} λ + c_n \)
using the recursion
\( B_0 = I,\; c_k = -\frac{1}{k}\,\mathrm{tr}(A B_{k-1}),\; B_k = A B_{k-1} + c_k I. \)

In [ ]:
def leverrier_faddeev(A):
    A = to_sym(A)
    n = A.shape[0]
    I = sp.eye(n)
    B = I
    cs = []
    for k in range(1, n+1):
        ck = -sp.Trace(A*B) / k
        cs.append(sp.simplify(ck))
        B = A*B + ck*I
    # Polynomial: λ^n + c1 λ^{n-1} + ... + c_n
    lam = sp.symbols('λ')
    p = lam**n
    for i, ck in enumerate(cs, start=1):
        p += ck * lam**(n - i)
    return sp.expand(p), cs  # polynomial expression, coefficients

## 5) Strict diagonal dominance (SDD) and Gershgorin discs
- A matrix is **strictly diagonally dominant** if \(|a_{ii}| > \sum_{j\ne i}|a_{ij}|\) for all i.
- SDD matrices are **nonsingular** (Levy–Desplanques). A Gershgorin view: zero cannot lie in any disc.
We include utility checks and an optional plot for Gershgorin discs.

In [ ]:
def is_strictly_diagonally_dominant(A):
    A = np.array(A, dtype=float)
    n = A.shape[0]
    for i in range(n):
        diag = abs(A[i,i])
        off = np.sum(np.abs(A[i,:])) - diag
        if not (diag > off):
            return False
    return True

def gershgorin_disks(A):
    A = np.array(A, dtype=float)
    n = A.shape[0]
    centers = A.diagonal()
    radii = [np.sum(np.abs(A[i,:])) - abs(A[i,i]) for i in range(n)]
    return centers, radii

def plot_gershgorin(A, title="Gershgorin Discs"):
    centers, radii = gershgorin_disks(A)
    fig, ax = plt.subplots(figsize=(6,6))
    # Plot each disc as a circle in the complex plane (real axis only for centers here)
    for c, r in zip(centers, radii):
        circle = Circle((c, 0.0), r, fill=False)
        ax.add_patch(circle)
        ax.plot([c], [0.0], marker='o')
    ax.axhline(0.0)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(title)
    plt.show()

---

## 6) Quick demos

Run these cells to see the functions in action.

In [ ]:
# Demo 1: Basic eigen computations
A = sp.Matrix([[2, 1, 0],
               [0, 2, 0],
               [0, 0, 3]])
print("A ="); sp.pprint(A)

print("\nCharacteristic polynomial (SymPy):")
lam = sp.symbols('λ')
cp = charpoly_sym(A)
sp.pprint(cp)

print("\nEigenvalues / eigenvectors (SymPy):")
for ev, mult, basis in eig_sympy(A):
    print("λ =", ev, "| alg mult =", mult, "| eigenbasis vectors:", basis)

print("\nVerify trace/det identities:")
sp.pprint(verify_trace_det(A))

print("\nPowers/inverse eigenvalues:")
sp.pprint(eigs_of_power_and_inverse(A, k=3))

In [ ]:
# Demo 2: Multiplicities & diagonalization check
B = sp.Matrix([[4, 1, 0],
               [0, 4, 0],
               [0, 0, 1]])
sp.pprint(algebraic_geometric_multiplicities(B))
P, D = diagonalize_if_possible(B)
print("\nDiagonalizable? ->", P is not None)
if P is not None:
    print("P ="); sp.pprint(P)
    print("D ="); sp.pprint(D)
    print("Check P^{-1} A P == D:", sp.simplify(P.inv()*B*P - D) == sp.zeros(*B.shape))

In [ ]:
# Demo 3: Leverrier–Faddeev vs SymPy charpoly
np.random.seed(0)
C = sp.Matrix(np.random.randint(-3, 4, size=(3,3)))
print("C ="); sp.pprint(C)
p_LF, cs = leverrier_faddeev(C)
print("\nLeverrier–Faddeev polynomial p(λ):"); sp.pprint(p_LF)
print("coeffs (c1..c_n):", cs)
print("\nSymPy charpoly:"); sp.pprint(charpoly_sym(C).as_expr())

In [ ]:
# Demo 4: Strict diagonal dominance & Gershgorin
D = np.array([[5, -1,  0],
              [2,  6,  1],
              [0,  1,  4]], dtype=float)
print("D is strictly diagonally dominant? ->", is_strictly_diagonally_dominant(D))

# Plot Gershgorin discs (no explicit colors set)
plot_gershgorin(D, title="Gershgorin Discs for D")

In [ ]:
# Demo 5: 2x2 rotation (complex eigenvalues) and a real symmetric example
theta = math.pi/4  # 45 degrees
R = np.array([[math.cos(theta), -math.sin(theta)],
              [math.sin(theta),  math.cos(theta)]], dtype=float)

vals_R, vecs_R = eig_numpy(R)
print("Rotation matrix R eigenvalues (complex on unit circle):", vals_R)

S = np.array([[2, 1],
              [1, 3]], dtype=float)
vals_S, vecs_S = eig_numpy(S)
print("Symmetric S eigenvalues (real) ->", vals_S)
print("Check orthogonality of eigenvectors numerically:", np.allclose(vecs_S.T @ vecs_S, np.eye(2)))